In [1]:
import requests 
from bs4 import BeautifulSoup
import re

In [29]:
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import os

chromedriver = "/Users/elaine/Dropbox/vcfriendfinder/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


In [49]:
import json
coData = []
with open('SJ_salaries.json', 'r') as f:
    for line in f:
        q = json.dumps(line, encoding='utf8')
        jfile = json.loads(q)
        

In [92]:
x = json.loads(jfile[323010:418278])

In [97]:
len(jfile)

548003

In [98]:
print len(cisco)
ppl = x[x.keys()[0]].keys()

1152


In [6]:
# collect one linkedin url for each jorb
from pygoogle import pygoogle

In [95]:
import time
chrome_options = Options()
chrome_options.add_extension('/Users/elaine/Dropbox/hook.crx')

browser = webdriver.Chrome(chromedriver, chrome_options=chrome_options)

In [15]:
# for each linkedin profile
def dump_profiles(urls, pos, browser):
    profiles = []
    for url in urls:
        browser.get(url)
        time.sleep(1)
        try:
            ent = browser.find_element_by_tag_name("body")
            profile = ent.find_element_by_id("background").get_attribute('innerHTML')
            profiles.append(profile)
        except:
            continue

        #for each title, save company, title, and array of profiles to disk
    pdict = {'company': 'Google', 'position': pos, 'profiles': profiles}
    parse_li(pdict)

In [ ]:
for goog in ppl[123:]:
    search = "https://www.linkedin.com/title/ " + goog + " at Cisco Systems"
    g = pygoogle(search)
    g.pages = 1
    try:
        li_url = g.get_urls()[0]
    except:
        print goog
        print ppl.index(goog)
    time.sleep(30)

    r = requests.get(li_url)
    profiles = BeautifulSoup(r.content).findAll('li',{'class': 'item-container'})
    urls = []
    for p in profiles:
        href = p.find('a')['href']
        urls.append(href)
    goog = goog.replace("/","-")
    li_urls.append({'position':goog, 'company':'Cisco Systems', 'profiles': urls})
    # get these later
    dump_profiles(urls, goog, browser)

In [14]:
#parse linkedin json so it's not so onerous
# one document per person

def parse_li(x):
    boxes =['background-summary-container','background-experience-container',
        'background-languages-container', 'background-skills-container',
       'background-education-container','background-honors-container']
    cleanprofiles = []
    for p in x['profiles']:
        content = ""
        for b in boxes:
            try:
                divs = BeautifulSoup(p).find('div', {'id': b})
                for script in divs(["script", "style"]):
                    script.extract()    # rip it out   
                for d in divs.findAll('div'):
                    content += " ".join(d.strings) + " "
            except:
                continue
        cleanprofiles.append(content)
    x['profiles']=cleanprofiles
    x['position'].replace("/","-")
    with open('data/'+x['position']+'.json','w') as f:
        json.dump(x, f)

In [101]:
len(ppl)


1152

In [ ]:
with open('data/Market Research-Business Development Analyst.json','w') as f:
    json.dump(x, f)